### Supermarket Simulation Using Markov Chains

In [1]:
import pandas as pd
import seaborn as sns

### 1. EDA

In [2]:
df = pd.read_csv('_RES/monday.csv', sep=';')

In [3]:
df.shape

(4884, 3)

In [4]:
df.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [5]:
df.tail()

,timestamp,customer_no,location
4879,2019-09-02 21:49:00,1442,checkout
4880,2019-09-02 21:49:00,1444,checkout
4881,2019-09-02 21:49:00,1445,dairy
4882,2019-09-02 21:50:00,1446,dairy
4883,2019-09-02 21:50:00,1447,fruit


In [6]:
df.describe()

,customer_no
count,4884.000000
mean,718.274365
std,411.839636
min,1.000000
25%,366.000000
50%,720.000000
75%,1070.000000
max,1447.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    4884 non-null   object
 1   customer_no  4884 non-null   int64 
 2   location     4884 non-null   object
dtypes: int64(1), object(2)
memory usage: 114.6+ KB


#### Create datetime index

In [8]:
datetime = pd.to_datetime(df['timestamp'])
df.index = pd.DatetimeIndex(datetime)
df = df.drop(['timestamp'], axis=1)

In [9]:
df.head()

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
2019-09-02 07:04:00,4,dairy
2019-09-02 07:04:00,5,spices


#### Calculate the total number of customers in each section

In [10]:
df.groupby(by=['location']).nunique()

,customer_no,location
location,,
checkout,1437,1
dairy,720,1
drinks,661,1
fruit,827,1
spices,584,1


#### Calculate the total number of customers in each section over time

Showing how many customers are present on how many locations over time

In [11]:
df.groupby(by=['timestamp']).nunique()

,customer_no,location
timestamp,,
2019-09-02 07:03:00,2,1
2019-09-02 07:04:00,6,3
2019-09-02 07:05:00,4,3
2019-09-02 07:06:00,8,5
2019-09-02 07:07:00,4,4
...,...,...
2019-09-02 21:46:00,6,4
2019-09-02 21:47:00,6,3
2019-09-02 21:48:00,7,4


Showing how many customers are present at individual locations over time

In [12]:
df.groupby(['timestamp', 'location']).count().unstack(level=1)

customer_no                          
location               checkout dairy drinks fruit spices
timestamp                                                
2019-09-02 07:03:00         NaN   2.0    NaN   NaN    NaN
2019-09-02 07:04:00         NaN   2.0    NaN   1.0    3.0
2019-09-02 07:05:00         2.0   1.0    1.0   NaN    NaN
2019-09-02 07:06:00         2.0   2.0    1.0   2.0    1.0
2019-09-02 07:07:00         1.0   1.0    1.0   NaN    1.0
...                         ...   ...    ...   ...    ...
2019-09-02 21:46:00         1.0   NaN    2.0   1.0    2.0
2019-09-02 21:47:00         2.0   2.0    NaN   2.0    NaN
2019-09-02 21:48:00         2.0   2.0    2.0   NaN    1.0
2019-09-02 21:49:00         2.0   1.0    NaN   2.0    2.0
2019-09-02 21:50:00         NaN   1.0    NaN   1.0    NaN

[874 rows x 5 columns]

Showing grouped information on location over time for individual customers

In [13]:
df.groupby(['customer_no', 'timestamp']).sum()

location
customer_no timestamp                    
1           2019-09-02 07:03:00     dairy
            2019-09-02 07:05:00  checkout
2           2019-09-02 07:03:00     dairy
            2019-09-02 07:06:00  checkout
3           2019-09-02 07:04:00     dairy
...                                   ...
1444        2019-09-02 21:48:00    spices
            2019-09-02 21:49:00  checkout
1445        2019-09-02 21:49:00     dairy
1446        2019-09-02 21:50:00     dairy
1447        2019-09-02 21:50:00     fruit

[4884 rows x 1 columns]

#### Upsampling Customers
There are missing values for some customers at specific times, which can be forward filled (assuming they've always just stayed at the previous location)

Let's start with customer 7.

In [14]:
df[df['customer_no'] == 7]

,customer_no,location
timestamp,,
2019-09-02 07:04:00,7,spices
2019-09-02 07:05:00,7,drinks
2019-09-02 07:09:00,7,spices
2019-09-02 07:11:00,7,fruit
2019-09-02 07:13:00,7,checkout


In [15]:
df[df['customer_no'] == 7].resample('T').ffill()

,customer_no,location
timestamp,,
2019-09-02 07:04:00,7,spices
2019-09-02 07:05:00,7,drinks
2019-09-02 07:06:00,7,drinks
2019-09-02 07:07:00,7,drinks
2019-09-02 07:08:00,7,drinks
2019-09-02 07:09:00,7,spices
2019-09-02 07:10:00,7,spices
2019-09-02 07:11:00,7,fruit
2019-09-02 07:12:00,7,fruit


Of course, if we now want to do this for all customers at the same time using 'unstack' this gets problematic, because most of the timestamps already exist for one customer or the other. That's why we might need to do this with a for-loop after all ...

In [16]:
df.groupby(['timestamp', 'customer_no']).sum().unstack(level=1).resample('T').ffill()

location                                               \
customer_no              1         2         3      4         5       6      
timestamp                                                                    
2019-09-02 07:03:00     dairy     dairy       NaN    NaN       NaN     NaN   
2019-09-02 07:04:00       NaN       NaN     dairy  dairy    spices  spices   
2019-09-02 07:05:00  checkout       NaN       NaN    NaN  checkout   dairy   
2019-09-02 07:06:00       NaN  checkout  checkout    NaN       NaN     NaN   
2019-09-02 07:07:00       NaN       NaN       NaN    NaN       NaN     NaN   
...                       ...       ...       ...    ...       ...     ...   
2019-09-02 21:46:00       NaN       NaN       NaN    NaN       NaN     NaN   
2019-09-02 21:47:00       NaN       NaN       NaN    NaN       NaN     NaN   
2019-09-02 21:48:00       NaN       NaN       NaN    NaN       NaN     NaN   
2019-09-02 21:49:00       NaN       NaN       NaN    NaN       NaN     NaN   
2019-09-02 21:50:00       NaN       NaN       NaN    NaN       NaN     NaN   

                                                     ...                   \
customer_no            7         8      9      10    ...      1438   1439   
timestamp                                            ...                    
2019-09-02 07:03:00     NaN       NaN    NaN    NaN  ...       NaN    NaN   
2019-09-02 07:04:00  spices     fruit    NaN    NaN  ...       NaN    NaN   
2019-09-02 07:05:00  drinks       NaN    NaN    NaN  ...       NaN    NaN   
2019-09-02 07:06:00     NaN       NaN  dairy  fruit  ...       NaN    NaN   
2019-09-02 07:07:00     NaN  checkout    NaN    NaN  ...       NaN    NaN   
...                     ...       ...    ...    ...  ...       ...    ...   
2019-09-02 21:46:00     NaN       NaN    NaN    NaN  ...    spices  fruit   
2019-09-02 21:47:00     NaN       NaN    NaN    NaN  ...  checkout    NaN   
2019-09-02 21:48:00     NaN       NaN    NaN    NaN  ...       NaN    NaN   
2019-09-02 21:49:00     NaN       NaN    NaN    NaN  ...       NaN    NaN   
2019-09-02 21:50:00     NaN       NaN    NaN    NaN  ...       NaN    NaN   

                                                                              \
customer_no            1440    1441      1442   1443      1444   1445   1446   
timestamp                                                                      
2019-09-02 07:03:00     NaN     NaN       NaN    NaN       NaN    NaN    NaN   
2019-09-02 07:04:00     NaN     NaN       NaN    NaN       NaN    NaN    NaN   
2019-09-02 07:05:00     NaN     NaN       NaN    NaN       NaN    NaN    NaN   
2019-09-02 07:06:00     NaN     NaN       NaN    NaN       NaN    NaN    NaN   
2019-09-02 07:07:00     NaN     NaN       NaN    NaN       NaN    NaN    NaN   
...                     ...     ...       ...    ...       ...    ...    ...   
2019-09-02 21:46:00  drinks     NaN       NaN    NaN       NaN    NaN    NaN   
2019-09-02 21:47:00     NaN   dairy     fruit  fruit       NaN    NaN    NaN   
2019-09-02 21:48:00     NaN  drinks       NaN  dairy    spices    NaN    NaN   
2019-09-02 21:49:00  spices  spices  checkout    NaN  checkout  dairy    NaN   
2019-09-02 21:50:00     NaN     NaN       NaN    NaN       NaN    NaN  dairy   

                            
customer_no           1447  
timestamp                   
2019-09-02 07:03:00    NaN  
2019-09-02 07:04:00    NaN  
2019-09-02 07:05:00    NaN  
2019-09-02 07:06:00    NaN  
2019-09-02 07:07:00    NaN  
...                    ...  
2019-09-02 21:46:00    NaN  
2019-09-02 21:47:00    NaN  
2019-09-02 21:48:00    NaN  
2019-09-02 21:49:00    NaN  
2019-09-02 21:50:00  fruit  

[888 rows x 1447 columns]

In [17]:
# df.groupby(by=[lambda dt: dt.strftime('%Y-%m-%d'),'browser']).size().groupby(level=0).apply(
#     lambda x: 100 * x / x.sum()
# ).unstack().plot(kind='bar',stacked=True,legend='reverse',width=0.8)
# 
# plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
# plt.show()